In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms

from datetime import datetime
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets

from resnet import ResNet20

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [2]:
class Feather:
    def __init__(self, gth, theta):
        self.gth = gth
        self.theta = theta
    
    def forward(self, w):
        return Feather_aux.apply(w, self.gth, self.theta)
        


class Feather_aux(torch.autograd.Function):
    @staticmethod
    def forward(ctx, w, gth, theta):
             
        ctx.aux = torch.where(torch.abs(w) > gth, 1.0, theta)
     
        p = 3
        diff = torch.abs(w)**p - gth**p 
        w_masked = torch.where(diff > 0, torch.sign(w)*(diff)**(1/p), 0.0)   
        
        return w_masked

    @staticmethod
    def backward(ctx, g):
        
        g = ctx.aux*g           
        return g, None, None 
    

class SparseConv(nn.Module):
    def __init__(self, conv, feather):
        super(SparseConv, self).__init__()
        self.conv = conv
        self.feather = feather

    def forward(self, x):

        w = self.conv.weight
        b = self.conv.bias
        stride = self.conv.stride
        padding = self.conv.padding
        groups = self.conv.groups


        if self.feather.gth > 0:
            w = self.feather.forward(w)

        out = F.conv2d(x, w, bias=b, padding=padding, stride=stride, groups=groups)
        return out

class SparseFc(nn.Module):
    def __init__(self, fc, feather):
        super(SparseFc, self).__init__()
        self.fc = fc
        self.feather = feather
 
    def forward(self, x):

        w = self.fc.weight
        b = self.fc.bias
                
        if self.feather.gth > 0:            
            w = self.feather.forward(w)
            
        out = F.linear(x, w, bias=b)
        return out


def iter_sparsify(m, feather, pthres=0):
    for name, child in m.named_children():
        iter_sparsify(child, feather, pthres)
        if type(child) == nn.Conv2d:
            nw = (child.in_channels * child.out_channels * child.kernel_size[0] * child.kernel_size[1]) / child.groups
            if nw >= pthres:  
                slayer = SparseConv(child, feather)
                m.__setattr__(name, slayer)

              
        if type(child) == nn.Linear:
            nw = child.in_features * child.out_features
            if nw >= pthres:
                slayer = SparseFc(child, feather)
                m.__setattr__(name, slayer)


def iter_desparsify(m, feather):
    for name, child in m.named_children():
        iter_desparsify(child, feather)
        if type(child) == SparseConv:
            conv = child.conv
            w = conv.weight.data
            
            nw = feather.forward(w)
            conv.weight.data = nw

            m.__setattr__(name, conv)

        if type(child) == SparseFc:
            fc= child.fc
            w = fc.weight.data

            nw = feather.forward(w)
            fc.weight.data = nw

            m.__setattr__(name, fc)

 
def get_params(model):
    bn_ids =[]  
    modules = list(model.named_modules())      
    for n, layer in modules:
        if isinstance(layer,torch.nn.modules.batchnorm.BatchNorm2d): 
            bn_ids.append(id(layer.weight))
            bn_ids.append(id(layer.bias))
    
    params, params_nowd = [], []
    for name, p in model.named_parameters():
    
        if id(p) in bn_ids or 'bias' in name:
            params_nowd += [p]
        else:
            params += [p]
    return params, params_nowd


def get_prunable_weights_cnt(model):
    prunable_weights_cnt = 0
    temp_dims = [0]
    for name, layer in model.named_modules():
        if ('Sparse' in layer.__class__.__name__):
            if 'Conv' in layer.__class__.__name__ :
                w = layer.conv.weight
            elif 'Fc' in layer.__class__.__name__:
                w = layer.fc.weight
            else:
                print(" Not Recognized Sparse Module ")

            temp_dims.append(w.numel())         
            tnum = w.numel()
            prunable_weights_cnt += tnum
            
    idx_list = [0]
    for i in range(len(temp_dims)):
        idx_list.append(temp_dims[i] + idx_list[i])
            
    return prunable_weights_cnt, idx_list


def calc_thresh(w, ratio):
    w_sorted, _ = torch.sort(w)
    m = (len(w_sorted)-1)*ratio
    idx_floor, idx_ceil = int(np.floor(m)), int(np.ceil(m))
    v1, v2 = w_sorted[idx_floor], w_sorted[idx_ceil]
    thresh = v1 + (v2-v1)*(m-idx_floor)
    return thresh.item()



def get_global_thresh(model, device, st_batch, prunable_weights_cnt, idx_list):
    i = 1
    w_total = torch.empty(prunable_weights_cnt).to(device) 
    for name, layer in model.named_modules():
        if ('Sparse' in layer.__class__.__name__):
            if 'Conv' in layer.__class__.__name__ :
                w = layer.conv.weight.flatten().detach()            
            elif 'Fc' in layer.__class__.__name__:
                w = layer.fc.weight.flatten().detach() 

            w_total[idx_list[i] : idx_list[i+1]] = w
            i +=1
         
    global_thresh = calc_thresh(torch.abs(w_total), st_batch)
    
    return global_thresh


def pruning_scheduler(final_rate, nbatches, ntotalsteps, t1):
    
    kf = final_rate
    t1 = t1*nbatches   
    t2 = int(np.floor(ntotalsteps*0.5))

    t = np.arange(t1,t2)
    k = np.hstack(( np.zeros(t1), ( kf - kf*(1-(t-t1)/(t2-t1))**3), kf*np.ones(ntotalsteps-t2) )) 

    return k
 
def get_theta(final_rate):
    if final_rate > 0.95:
        theta = 0.5
    else:
        theta = 1.0
    return theta


class Pruner:  
    def __init__(self, model, device, final_rate,  nbatches, epochs, pthres=0, t1=0):

        theta = get_theta(final_rate)
        self.ntotalsteps = nbatches*epochs
        self.step_idx = 0
        
        self.feather = Feather(gth=0.0, theta=theta)     
        self.device = device
        self.t1 = t1
        
        self.model = model 
           
        iter_sparsify(m=self.model, feather=self.feather, pthres=pthres)     
        prunable_weights_cnt, idx_list = get_prunable_weights_cnt(self.model)
        
        self.prunable_weights_cnt = prunable_weights_cnt
        self.idx_list = idx_list
        
        pscheduler = pruning_scheduler(final_rate, nbatches, self.ntotalsteps, self.t1)
        self.pscheduler = pscheduler
        
    def update_thresh(self, end_of_batch=False):       
        idx = self.step_idx
        if end_of_batch:
            idx -=1
        st_batch = self.pscheduler[idx]
        
        new_gth = 0.0
        if st_batch > 0:
            new_gth = get_global_thresh(self.model, self.device, st_batch, self.prunable_weights_cnt, self.idx_list)

        self.feather.gth = new_gth 
        if not end_of_batch:
            self.step_idx += 1 
        
    def print_sparsity(self):   
        local_zeros_cnt = 0
        for name, layer in self.model.named_modules():
            if ('Sparse' in layer.__class__.__name__):
                if 'Conv' in layer.__class__.__name__ :
                    w = layer.conv.weight
                elif 'Fc' in layer.__class__.__name__:
                    w = layer.fc.weight
                else:
                    print(" Not Recognized Sparse Module ")

                th = self.feather.gth 
                nw = F.hardshrink(w, th)
                      
                tsparsity = (nw == 0).float().sum().item()
                local_zeros_cnt += tsparsity
        
                tnum = nw.numel()
    
                print(f'{name}'.ljust(40), f'#w: {int(tnum)}'.ljust(11), f'| sparsity: {round(100.0 * tsparsity / tnum, 2)}%'.ljust(18))  
                  
        return 100 * float(local_zeros_cnt) / float(self.prunable_weights_cnt)
    
    def desparsify(self):
        iter_desparsify(self.model, self.feather)



In [3]:
hyperparameters = {
    'sparsity_type': "feather", # nm / entropy / feather / spartan / ses / base (no sparsity)
    'epochs': 200,
    'lr': 0.1,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'batch_size': 391,
}

In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.201]),
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.201]),
])


train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=hyperparameters['batch_size'], shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=hyperparameters['batch_size'], shuffle=False, num_workers=2)

In [5]:
sparsity_type = hyperparameters['sparsity_type']
output_base_path = "/home/sg666/Class/ECE661/outputs"
sparsity_folder_path = os.path.join(output_base_path, sparsity_type)

hyperparameter_str = f"epochs_{hyperparameters['epochs']}_lr_{hyperparameters['lr']}_momentum_{hyperparameters['momentum']}_wd_{hyperparameters['weight_decay']}_batch_{hyperparameters['batch_size']}"
output_folder = os.path.join(sparsity_folder_path, hyperparameter_str)

os.makedirs(output_folder, exist_ok=True)

hyperparameter_file = os.path.join(output_folder, 'hyperparameters.txt')
with open(hyperparameter_file, 'w') as f:
    for key, value in hyperparameters.items():
        f.write(f"{key}: {value}\n")

In [6]:
resnet20_model = ResNet20()
resnet20_model.to(device)

pruner = Pruner(resnet20_model, device, final_rate=0.95, nbatches=hyperparameters['batch_size'], epochs=hyperparameters['epochs'])

In [7]:
def train(model, train_loader, criterion, optimizer, epoch, log_file):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}", ncols=100)
    
    for batch_idx, (inputs, targets) in enumerate(pbar):
        pruner.update_thresh()

        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        total += targets.size(0)

        pbar.set_postfix(loss=running_loss/(batch_idx+1), accuracy=100.0 * correct / total)

    avg_loss = running_loss / len(train_loader)
    accuracy = 100.0 * correct / total

    sparsity = pruner.print_sparsity()
    log_file.write(f'Epoch [{epoch+1}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%, Sparsity: {sparsity:.2f}%\n')
    
    pruner.update_thresh(end_of_batch=True)

    return avg_loss, accuracy

In [8]:
def test(model, test_loader, criterion, log_file):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0

    pbar = tqdm(test_loader, desc="Testing", ncols=100)
    
    with torch.no_grad():
        for inputs, targets in pbar:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)

            pbar.set_postfix(loss=test_loss/(total + inputs.size(0)), accuracy=100.0 * correct / total)

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100.0 * correct / total
    log_file.write(f'Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.2f}%\n')

    return avg_test_loss, accuracy


In [9]:
current_learning_rate = hyperparameters['lr']
DECAY = 0.1
DECAY_EPOCHS = [100, 150]
EPOCHS = 200

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet20_model.parameters(), lr=hyperparameters['lr'], 
                      momentum=hyperparameters['momentum'], weight_decay=hyperparameters['weight_decay'])

log_file_path = os.path.join(output_folder, 'training_log.txt')
with open(log_file_path, 'w') as log_file:
    log_file.write(f"Training started at {datetime.now()}\n")

    best_accuracy = 0.0

    for epoch in range(hyperparameters['epochs']):
        train_loss, train_accuracy = train(resnet20_model, train_loader, criterion, optimizer, epoch, log_file)
        test_loss, test_accuracy = test(resnet20_model, test_loader, criterion, log_file)

        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            model_checkpoint_path = os.path.join(output_folder, f"best_model.pth")
            torch.save(resnet20_model.state_dict(), model_checkpoint_path)
            print(f"Saved best model at epoch {epoch+1} with accuracy: {best_accuracy:.2f}%")

        if epoch+1 in DECAY_EPOCHS:
            current_learning_rate = current_learning_rate * DECAY
            for param_group in optimizer.param_groups:
                param_group['lr'] = current_learning_rate
            print("Current learning rate has decayed to %f" %current_learning_rate)


    log_file.write(f"Training completed at {datetime.now()}\n")
    log_file.write(f"Best model accuracy: {best_accuracy:.2f}%\n")


Epoch 1: 100%|██████████████████████████| 128/128 [00:06<00:00, 18.36it/s, accuracy=36.4, loss=1.69]


conv1                                    #w: 432     | sparsity: 0.46% 
residual_layers.0.conv1                  #w: 2304    | sparsity: 0.26% 
residual_layers.0.conv2                  #w: 2304    | sparsity: 0.48% 
residual_layers.1.conv1                  #w: 2304    | sparsity: 0.22% 
residual_layers.1.conv2                  #w: 2304    | sparsity: 0.56% 
residual_layers.2.conv1                  #w: 2304    | sparsity: 0.78% 
residual_layers.2.conv2                  #w: 2304    | sparsity: 0.43% 
residual_layers.3.conv1                  #w: 4608    | sparsity: 0.52% 
residual_layers.3.conv2                  #w: 9216    | sparsity: 0.62% 
residual_layers.3.residual.0             #w: 512     | sparsity: 0.2%  
residual_layers.4.conv1                  #w: 9216    | sparsity: 0.65% 
residual_layers.4.conv2                  #w: 9216    | sparsity: 0.89% 
residual_layers.5.conv1                  #w: 9216    | sparsity: 0.88% 
residual_layers.5.conv2                  #w: 9216    | sparsity:

Testing: 100%|█████████████████████████| 26/26 [00:01<00:00, 25.62it/s, accuracy=46.5, loss=0.00372]


Saved best model at epoch 1 with accuracy: 46.49%


Epoch 2: 100%|████████████████████████████| 128/128 [00:06<00:00, 20.47it/s, accuracy=56, loss=1.21]


...

Epoch 199: 100%|███████████████████████| 128/128 [00:06<00:00, 19.78it/s, accuracy=93.4, loss=0.189]


conv1                                    #w: 432     | sparsity: 48.38%
residual_layers.0.conv1                  #w: 2304    | sparsity: 82.07%
residual_layers.0.conv2                  #w: 2304    | sparsity: 81.21%
residual_layers.1.conv1                  #w: 2304    | sparsity: 82.86%
residual_layers.1.conv2                  #w: 2304    | sparsity: 81.99%
residual_layers.2.conv1                  #w: 2304    | sparsity: 78.65%
residual_layers.2.conv2                  #w: 2304    | sparsity: 82.64%
residual_layers.3.conv1                  #w: 4608    | sparsity: 79.69%
residual_layers.3.conv2                  #w: 9216    | sparsity: 85.87%
residual_layers.3.residual.0             #w: 512     | sparsity: 60.55%
residual_layers.4.conv1                  #w: 9216    | sparsity: 87.9% 
residual_layers.4.conv2                  #w: 9216    | sparsity: 89.1% 
residual_layers.5.conv1                  #w: 9216    | sparsity: 86.12%
residual_layers.5.conv2                  #w: 9216    | sparsity:

Epoch 200: 100%|███████████████████████| 128/128 [00:06<00:00, 20.41it/s, accuracy=93.4, loss=0.191]


conv1                                    #w: 432     | sparsity: 49.07%
residual_layers.0.conv1                  #w: 2304    | sparsity: 82.29%
residual_layers.0.conv2                  #w: 2304    | sparsity: 81.34%
residual_layers.1.conv1                  #w: 2304    | sparsity: 83.07%
residual_layers.1.conv2                  #w: 2304    | sparsity: 82.55%
residual_layers.2.conv1                  #w: 2304    | sparsity: 78.82%
residual_layers.2.conv2                  #w: 2304    | sparsity: 82.94%
residual_layers.3.conv1                  #w: 4608    | sparsity: 80.14%
residual_layers.3.conv2                  #w: 9216    | sparsity: 85.94%
residual_layers.3.residual.0             #w: 512     | sparsity: 60.74%
residual_layers.4.conv1                  #w: 9216    | sparsity: 88.11%
residual_layers.4.conv2                  #w: 9216    | sparsity: 89.21%
residual_layers.5.conv1                  #w: 9216    | sparsity: 86.15%
residual_layers.5.conv2                  #w: 9216    | sparsity:

Testing: 100%|████████████████████████| 26/26 [00:00<00:00, 26.39it/s, accuracy=88.2, loss=0.000964]
